In [1]:
import os
import sys
import random
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import cv2
import itertools
from pathlib import Path
from glob import glob
from tqdm.notebook import tqdm
from multiprocessing import Pool

from scipy.optimize import minimize, minimize_scalar
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KDTree

from external_lib.NFLlib.score import NFLAssignmentScorer, check_submission
from external_lib.NFLlib.features import add_track_features


In [2]:
n_debug_samples = 120
random_state = 42
CONF_THRE = 0.3
max_iter = 1000
DIG_STEP = 3
DIG_MAX = 42
base_dir = "/work/data/input/nfl-health-and-safety-helmet-assignment"

debug = True


paramater = {};
paramater["n_debug_samples"] = n_debug_samples
paramater["random_state"] = random_state
paramater["CONF_THRE"] = CONF_THRE
paramater["max_iter"] = max_iter
paramater["DIG_STEP"] = DIG_STEP
paramater["DIG_MAX"] = random_state


In [3]:
def add_cols(df):
    df['game_play'] = df['video_frame'].str.split('_').str[:2].str.join('_')
    if not 'video' in df.columns:
        df['video'] = df['video_frame'].str.split('_').str[:3].str.join('_') + '.mp4'
    return df


# Reading data
DEBUG = (len(glob(f"{base_dir}/test/*" )) == 6)

if DEBUG:
    tracking = pd.read_csv(f"{base_dir}/train_player_tracking.csv")
    helmets = pd.read_csv(f"{base_dir}/train_baseline_helmets.csv")
else:
    tracking = pd.read_csv(f"{base_dir}/test_player_tracking.csv") 
    helmets = pd.read_csv(f"{base_dir}/test_baseline_helmets.csv")
    
labels = pd.read_csv(f"{base_dir}/train_labels.csv")
sub = pd.read_csv(f"{base_dir}/sample_submission.csv")

# processing data
tracking = add_track_features(tracking)
helmets = add_cols(helmets)
labels = add_cols(labels)


#view_mode = "Endzone"
#helmets = helmets[helmets["video_frame"].str.contains(view_mode)]
#labels = labels[labels["video_frame"].str.contains(view_mode)]

# sampling Data
if False:
    sample_videos = labels['video'].drop_duplicates().sample(n_debug_samples, random_state=42).tolist()
    #sample_videos = labels['video'].drop_duplicates().tolist()
    sample_gameplays = ['_'.join(x.split('_')[:2]) for x in sample_videos]
    tracking = tracking[tracking['game_play'].isin(sample_gameplays)]
    helmets = helmets[helmets['video'].isin(sample_videos)]
    labels = labels[labels['video'].isin(sample_videos)]

print(tracking.shape, helmets.shape, labels.shape)

(333811, 18) (1225172, 8) (952087, 15)


In [4]:
from lib.math.coodinate import generate_rotation_maxrix

In [5]:
depth_id = 0;
horizon_id = 1;
vertical_id = 2;

In [6]:
def custom_convert_3d_to_2d(points, from_point:np.array,to_point:np.array):
    """
    3次元空間を２次元空間へ投影させる。
    

    args:
        points: 3次元空間内の点（x,y,z）
        from_point: 視点（どこから見るか）
        to_point: 視心（どこを視るか）

    Returns:
        pd.DataFrame
        
        列ラベル
            奥行き: depth
            水平: horizon
            垂直: vertical        
        個数：pointsのレコード数と等しい        
    """

    a = np.linalg.norm(to_point[:2]-from_point[:2],ord=2)
    b = np.linalg.norm(to_point-from_point,ord=2)
    
    if(a<1e-6):
        sin_theta = 0
        cos_theta = 1
    else:
        tmp_a = (to_point - from_point)/a
        sin_theta = tmp_a[1]
        cos_theta = tmp_a[0];
                    
    if (b<1e-6):
        sin_phi = 0;                
        cos_phi = 1;                
    else:
        tmp_b = (to_point - from_point)/b
        sin_phi = tmp_b[2];        
        cos_phi = a/b
    

    
    rotmat = [
        [cos_phi*cos_theta, cos_phi*sin_theta, sin_phi],
        [-1*sin_theta, cos_theta,0],
        [-1*sin_phi*cos_theta, -sin_phi*sin_theta, cos_phi]
    ]
                    
    #x_mat = generate_rotation_maxrix(angle = 180,axis="x",isDeg=True);
    #y_mat = generate_rotation_maxrix(angle = 180,axis="y",isDeg=True);
    #z_mat = generate_rotation_maxrix(angle = 180,axis="z",isDeg=True);

    ret_df = (rotmat@(points-from_point).T).T
            
    #ret_df = ret_df.rename(columns = {0:"depth",1:"horizon",2:"vertical"})
    ret_df[:,1] = ret_df[:,1]*-1
    
    
    
    return ret_df;


In [20]:
def find_nearest(arr: np.array, value: int):
    arr, value = arr.astype(int), int(value)
    idx = np.abs(arr - value).argmin()
    return arr[idx]

def normalize_arr(arr: np.array):
    _mean = np.mean(arr, axis=0)
    out_arr = arr - _mean
    _norm = np.linalg.norm(out_arr)
    out_arr = out_arr / _norm
    return out_arr

def rotate_arr(u, t):
    t = np.deg2rad(t)
    R = np.array([
        [np.cos(t), -np.sin(t)],
        [np.sin(t),  np.cos(t)]
    ])
    return  (R@u.T).T




def custom_rotate_arr(u,view, t, isDeg=True):
    if isDeg:
        t = np.deg2rad(t);
        
    if(view == "Endzone"):
        to_Point = np.array([60,0,0])
        from_point = np.array([60,0.5*t,40])
                
    elif(view == "Sideline"):
        to_Point = np.array([60,53.3/2,0])
        from_point = [60 + t,71,40]        
    else:
        raise Exception(f"You can specify only Endzone or Sideline, not {view}");
        
    ret_u = custom_convert_3d_to_2d(u,from_point=from_point,to_point=to_Point)
    
    
    return ret_u


    

def mapping(h_arr: np.array, t_arr: np.array,view):
    depth_id = 0;
    horizon_id = 1;
    vertical_id = 2;

    
    
    out_norm = float('INF')
    out_idx = None
    out_x = None
    out_dig = None
    
    tree = KDTree(t_arr)
    for i in range(len(t_arr)):
        dist, idx = tree.query([t_arr[i]], k=len(h_arr))
        idx = np.sort(idx[0])
        norm_t_arr = normalize_arr(t_arr[idx])
        
        
        def opt_rot(dig):
            #rot_t_arr = rotate_arr(norm_t_arr, dig)
            #print()
            
            if (view == "Endzone"):
                arr = custom_rotate_arr(norm_t_arr,view,dig);                
                rot_t_arr = arr[:,[vertical_id,horizon_id]]
            elif (view == "Sideline"):
                arr = custom_rotate_arr(norm_t_arr,view,dig);
                rot_t_arr = arr[:,[horizon_id,depth_id]]
                
            else:
                raise Exception(f"unknown view mode {view}");
                        
            
            return np.linalg.norm(np.sort(rot_t_arr[:, 0])-h_arr[:, 0])
                                
        result = minimize_scalar(opt_rot, bounds=[-DIG_MAX,DIG_MAX], method='bounded') 
        if out_norm > result.fun:
            out_norm = result.fun
            out_idx = idx                
            out_dig = result.x
                                
            if (view == "Endzone"):
                out_x = custom_rotate_arr(norm_t_arr,view, out_dig)[:,vertical_id]
            elif (view == "Sideline"):
                out_x = custom_rotate_arr(norm_t_arr,view, out_dig)[:,horizon_id]
            else:
                raise Exception(f"unknown view mode {view}");
                    
                        
    return out_idx, out_x

def main(args: pd.DataFrame):
    video_frame, subhelmets = args
    gameKey, playID, view, frame = video_frame.split('_')
    gameKey, playID, frame = int(gameKey), int(playID), int(frame)

    # get nearest-frame
    _index = (tracking['gameKey']==gameKey) & (tracking['playID']==playID)
    subtracking = tracking[_index].copy()
    est_frame = find_nearest(subtracking["est_frame"].values, frame)
    subtracking = subtracking[subtracking['est_frame']==est_frame]
    subtracking = subtracking.reset_index(drop=True)

    if view == 'Endzone':
        subtracking[['x', 'y']] = subtracking[['y', 'x']].values

    # normalizing 
    subhelmets = subhelmets[subhelmets['conf']>CONF_THRE].copy()

    if len(subhelmets) > len(subtracking):
        subhelmets = subhelmets.tail(len(subtracking))

    subhelmets['x'] = subhelmets['left'] + subhelmets['width'] // 2
    subhelmets['y'] = subhelmets['top'] + subhelmets['height'] // 2
    subhelmets[['norm_x', 'norm_y']] = normalize_arr(subhelmets[['x', 'y']].values)
    subhelmets = subhelmets.sort_values('norm_x').reset_index(drop=True)

    view = list(subhelmets["video_frame"].unique())[0].split("_")[2]
        
    # mapping tracking2helmets
    h_arr = subhelmets[['norm_x', 'norm_y']].values
    subtracking["z"] = 5
    t_arr = subtracking[['x', 'y',"z"]].values    
        
    out_idx, out_x = mapping(h_arr, t_arr,view)
    

    # helmets labeling 
    players = subtracking['player'].tolist()
    players = [p for i, p in enumerate(players) if i in out_idx]
    
    _pred = pd.DataFrame({'label': players, 'x': out_x}).sort_values('x')['label']
    subhelmets['label'] = _pred.values
    
    return subhelmets[['video_frame', 'left', 'width', 'top', 'height', 'label']]

In [8]:
from lib.multiprocess import exe_multiprocess_ret_as_list

In [12]:
ext_helmets = helmets[helmets["video_frame"].str.contains("57679_003316_Endzone")]

In [21]:
# multi processing
df_list = ext_helmets.groupby('video_frame')
submission_df = []


submission_df_list = exe_multiprocess_ret_as_list(main,df_list)

# submission
submission_df = pd.concat(submission_df_list).reset_index(drop=True)
submission_df.to_csv('submission.csv', index=False)

if DEBUG:
    scorer = NFLAssignmentScorer(labels[labels['video_frame'].isin(submission_df['video_frame'].unique())])
    score = scorer.score(submission_df)
    print(f'score: {round(score, 5)}')

  6%|▌         | 19/336 [00:03<00:51,  6.18it/s]Process ForkPoolWorker-19:
Process ForkPoolWorker-23:
Process ForkPoolWorker-21:
Process ForkPoolWorker-20:
Process ForkPoolWorker-22:
Process ForkPoolWorker-24:
Traceback (most recent call last):
  6%|▌         | 20/336 [00:04<01:08,  4.65it/s]Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/opt/conda/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/opt/conda/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.7/multip

  File "/opt/conda/lib/python3.7/site-packages/IPython/core/formatters.py", line 180, in format
    data = formatter(obj)
  File "/opt/conda/lib/python3.7/site-packages/numpy/core/_methods.py", line 47, in _sum
    return umr_sum(a, axis, dtype, out, keepdims, initial, where)
KeyboardInterrupt
  File "/opt/conda/lib/python3.7/site-packages/decorator.py", line 232, in fun
    return caller(func, *(extras + args), **kw)
  File "/opt/conda/lib/python3.7/site-packages/IPython/core/formatters.py", line 224, in catch_format_error
    r = method(self, *args, **kwargs)
  File "/opt/conda/lib/python3.7/site-packages/IPython/core/formatters.py", line 702, in __call__
    printer.pretty(obj)
  File "/opt/conda/lib/python3.7/site-packages/IPython/lib/pretty.py", line 394, in pretty
    return _repr_pprint(obj, self, cycle)
  File "/opt/conda/lib/python3.7/site-packages/IPython/lib/pretty.py", line 700, in _repr_pprint
    output = repr(obj)
  File "/tmp/ipykernel_1552057/1266988942.py", line 79, i

KeyboardInterrupt: 

In [ ]:
view = "Sideline"
#ext_sub = submission_df[submission_df["video_frame"].str.contains(view)]
#ext_sub
submission_df

## Endzone

In [ ]:
sub = submission_df[submission_df["video_frame"].str.contains("Endzone")]
if DEBUG:
    
    scorer = NFLAssignmentScorer(labels[labels['video_frame'].isin(sub['video_frame'].unique())])
    score = scorer.score(sub)
    print(f'score: {round(score, 5)}')

## Sideline

In [ ]:
sub = submission_df[submission_df["video_frame"].str.contains("Sideline")]
if DEBUG:
    
    scorer = NFLAssignmentScorer(labels[labels['video_frame'].isin(sub['video_frame'].unique())])
    score = scorer.score(sub)
    print(f'score: {round(score, 5)}')